In [ ]:
from ultralytics import YOLO
import pandas as pd
import os
import cv2
from tqdm import tqdm
import shutil
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
ANN_PATH = '/home/raid_storage/datasets/rosatom/full_dataset.csv'
FILTERED_DATASET = '/home/raid_storage/datasets/rosatom/filtered_dataset.csv'
DATASET_PATH = '/home/raid_storage/datasets/rosatom/'

In [ ]:
bbox_reg = 40

In [ ]:
df = pd.read_csv(FILTERED_DATASET).drop(['Unnamed: 0'], axis=1)
df = df[df['class'] != 0]
df

In [ ]:
h, w = 600, 960

In [ ]:
df['x_center'] = df.apply(lambda row: f'{round(int(row.x)/w, 5)}', axis=1)
df['y_center'] = df.apply(lambda row: f'{round(int(row.y)/h, 5)}', axis=1)
df['width'] = df.apply(lambda row: f'{round(2*bbox_reg/w, 5)}', axis=1)
df['height'] = df.apply(lambda row: f'{round(2*bbox_reg/h, 5)}', axis=1)
df['class'] = df.apply(lambda row: f'{int(row["class"])-1}', axis=1)

In [ ]:
dict_class = {0: 'не дефект',
              1: 'потертость',
              2: 'черная точка',
              3: 'плена',
              4: 'маркер',
              5: 'грязь',
              6: 'накол',
              7: 'н.д. накол',
              8: 'микровыступ',
              9: 'н.д. микровыступ',
              10: 'вмятина',
              11: 'мех.повреждение',
              12: 'риска',
              13: 'царапина с волчком'}

In [ ]:
for index, row in df.iterrows():
    stage = row['stage']
    if stage == 'train':
        dataset_yolo = '/home/raid_storage/isakov/hacks/notebooks/datasets/train/'
    else:
        dataset_yolo = '/home/raid_storage/isakov/hacks/notebooks/datasets/val/'
        
    filename = row.filename
    ann_filename = '_'.join(filename.split('/')[1:]).replace('bmp', 'txt')
    img_filename = '_'.join(filename.split('/')[1:])
    x_center = row['x_center']
    y_center = row['y_center']
    width = row['width']
    height = row['height']
    cls = row['class']

    with open(os.path.join(dataset_yolo, 'labels', ann_filename), 'a', encoding='utf-8') as ann_file:
        ann_file.write(f'{cls} {x_center} {y_center} {width} {height}\n')
    
    shutil.copy(os.path.join(DATASET_PATH, filename), os.path.join(dataset_yolo, 'images', img_filename))

In [ ]:
def plot_bounding_box(image_name):
    class_id_to_name = dict_class

    ann_name = image_name.replace('bmp', 'txt')
    ann_path = os.path.join(dataset_yolo, 'labels', ann_name)

    image_path = os.path.join(dataset_yolo, 'images', image_name)
    image = cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)
    h, w = image.shape[:2]
    print(h, w)

    with open(ann_path, 'r', encoding='utf-8') as ann:
        annotation_list = ann.read().split("\n")[:-1]
        annotation_list = [x.split(" ") for x in annotation_list]
        annotation_list = [[float(y) for y in x ] for x in annotation_list]

    annotations = np.array(annotation_list)
    transformed_annotations = np.copy(annotation_list)
    transformed_annotations[:,[1,3]] = annotations[:,[1,3]] * w
    transformed_annotations[:,[2,4]] = annotations[:,[2,4]] * h 

    transformed_annotations[:,1] = transformed_annotations[:,1] - (transformed_annotations[:,3] / 2)
    transformed_annotations[:,2] = transformed_annotations[:,2] - (transformed_annotations[:,4] / 2)
    transformed_annotations[:,3] = transformed_annotations[:,1] + transformed_annotations[:,3]
    transformed_annotations[:,4] = transformed_annotations[:,2] + transformed_annotations[:,4]
    
    for ann in transformed_annotations:
        obj_cls, x1, y1, x2, y2 = ann.astype(int)
        image = cv2.rectangle(image, (x1, y1), (x2, y2), (255, 0, 0), 3)

    plt.figure(figsize=(16, 8))
    plt.imshow(image)
    plt.show()

In [ ]:
plot_bounding_box('0_1538_frame0010.bmp')

In [ ]:
model = YOLO('yolov8m.yaml')  # build a new model from YAML
results = model.train(data='data.yaml', epochs=100, imgsz=960, device=[3], batch=24)

In [ ]:
model.export(format='engine', half=True, device=0)